In [47]:
# Import Dependencies

import pandas as pd
from sqlalchemy import create_engine, inspect, func
from config import password
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session

# Load and Read CSV's to Dataframes

In [124]:
# Game Info

game_csv = "archive/game.csv"
game_df = pd.read_csv(game_csv)
game_df_clean_1 = game_df[['game_id','season','type','away_team_id','home_team_id','outcome']].copy()
game_df_clean1 = game_df_clean_1.rename(columns = {'type':'game_type'})
game_df_clean = game_df_clean1.drop_duplicates('game_id')
game_df_clean

,game_id,season,game_type,away_team_id,home_team_id,outcome
0,2016020045,20162017,R,4,16,home win REG
1,2017020812,20172018,R,24,7,away win OT
2,2015020314,20152016,R,21,52,away win REG
3,2015020849,20152016,R,52,12,home win REG
4,2017020586,20172018,R,20,24,home win REG
...,...,...,...,...,...,...
26295,2018030413,20182019,P,6,19,away win REG
26297,2018030414,20182019,P,6,19,home win REG
26299,2018030415,20182019,P,19,6,away win REG
26301,2018030416,20182019,P,6,19,away win REG


In [55]:
# Team Info

team_info_csv = 'archive/team_info.csv'
team_info_df1 = pd.read_csv(team_info_csv)
team_info_df = team_info_df1.rename(columns = {'shortName': 'short_name',
                                              'teamName': 'team_name',
                                              'franchiseId': 'franchise_id'})
team_info_clean = team_info_df[['team_id','franchise_id', 'short_name','team_name']].copy()
team_info_clean.head()

,team_id,franchise_id,short_name,team_name
0,1,23,New Jersey,Devils
1,4,16,Philadelphia,Flyers
2,26,14,Los Angeles,Kings
3,14,31,Tampa Bay,Lightning
4,6,6,Boston,Bruins


In [57]:
# Player Info

player_info_csv = 'archive/player_info.csv'
player_info_df1 = pd.read_csv(player_info_csv)
player_info_df = player_info_df1.rename(columns = {'firstName': 'first_name',
                                                  'lastName': 'last_name',
                                                  'primaryPosition': 'primary_position'})
player_info_clean = player_info_df[['player_id','first_name','last_name','nationality','primary_position']]
player_info_clean.head()



,player_id,first_name,last_name,nationality,primary_position
0,8466148,Marian,Hossa,SVK,RW
1,8465058,Michal,Rozsival,CZE,D
2,8476906,Shayne,Gostisbehere,USA,D
3,8466285,Brian,Campbell,CAN,D
4,8470607,Brent,Seabrook,CAN,D


In [32]:
# Goalie Stats

game_goalie_csv = 'archive/game_goalie_stats.csv'
goalie_df = pd.read_csv(game_goalie_csv)
goalie_stats_clean = goalie_df[['game_id','player_id','team_id','shots','saves','decision']].copy()
goalie_stats_clean.head()




,game_id,player_id,team_id,shots,saves,decision
0,2016020045,8473607,4,16,12,NaN
1,2016020045,8473461,4,11,9,L
2,2016020045,8470645,16,27,23,W
3,2017020812,8468011,24,33,30,W
4,2017020812,8475215,7,33,29,L


In [58]:
# Penalty Stats

penalty_csv = 'archive/game_penalties.csv'
penalty_df1 = pd.read_csv(penalty_csv)
penalty_df = penalty_df1.rename(columns = {'penaltySeverity': 'penalty_severity',
                                          'penaltyMinutes': 'penalty_mins'})
penalty_df.head()

,play_id,penalty_severity,penalty_mins
0,2016020045_41,Minor,2
1,2016020045_101,Minor,2
2,2016020045_134,Minor,2
3,2016020045_174,Minor,2
4,2016020045_189,Minor,2


In [106]:
# Game skater Stats

game_skater_csv = 'archive/game_skater_stats.csv'
game_skater_df1 = pd.read_csv(game_skater_csv)
game_skater_df = game_skater_df1.rename(columns = {'penaltyMinutes': 'penalty_mins'})
game_skater_clean = game_skater_df[['game_id','player_id','team_id','assists','goals','shots','penalty_mins']].copy()
game_skater_clean


,game_id,player_id,team_id,assists,goals,shots,penalty_mins
0,2016020045,8468513,4,1,0,0,0
1,2016020045,8476906,4,1,0,4,2
2,2016020045,8474668,4,0,0,1,0
3,2016020045,8473512,4,3,0,0,0
4,2016020045,8471762,4,0,0,3,0
...,...,...,...,...,...,...,...
945825,2018030417,8475807,6,0,0,1,0
945826,2018030417,8475149,6,0,0,2,0
945827,2018030417,8475745,6,0,0,2,0
945828,2018030417,8478443,6,0,0,1,0


In [61]:
# Game Team Stats

game_team_csv = 'archive/game_teams_stats.csv'
game_team_df1 = pd.read_csv(game_team_csv)
game_team_df = game_team_df1.rename(columns = {'HoA': 'home_or_away'})
game_team_df.head()
game_team_clean = game_team_df[['game_id','team_id','home_or_away','won','goals','shots','hits','pim']].copy()
game_team_clean.head()



,game_id,team_id,home_or_away,won,goals,shots,hits,pim
0,2016020045,4,away,False,4.0,27.0,30.0,6.0
1,2016020045,16,home,True,7.0,28.0,20.0,8.0
2,2017020812,24,away,True,4.0,34.0,16.0,6.0
3,2017020812,7,home,False,3.0,33.0,17.0,8.0
4,2015020314,21,away,True,4.0,29.0,17.0,9.0


# L) Connect and load to Relational Database

In [107]:
connection_string = f"postgres:{password}@localhost:5432/NHL_stats"
engine = create_engine(f'postgresql://{connection_string}')


In [126]:
game_df_clean.to_sql(name = 'game_info', con = engine, if_exists = 'append', index = False)


IntegrityError: (psycopg2.errors.UniqueViolation) duplicate key value violates unique constraint "game_info_pkey"
DETAIL:  Key (game_id)=(2016020045) already exists.

[SQL: INSERT INTO game_info (game_id, season, game_type, away_team_id, home_team_id, outcome) VALUES (%(game_id)s, %(season)s, %(game_type)s, %(away_team_id)s, %(home_team_id)s, %(outcome)s)]
[parameters: ({'game_id': 2016020045, 'season': 20162017, 'game_type': 'R', 'away_team_id': 4, 'home_team_id': 16, 'outcome': 'home win REG'}, {'game_id': 2017020812, 'season': 20172018, 'game_type': 'R', 'away_team_id': 24, 'home_team_id': 7, 'outcome': 'away win OT'}, {'game_id': 2015020314, 'season': 20152016, 'game_type': 'R', 'away_team_id': 21, 'home_team_id': 52, 'outcome': 'away win REG'}, {'game_id': 2015020849, 'season': 20152016, 'game_type': 'R', 'away_team_id': 52, 'home_team_id': 12, 'outcome': 'home win REG'}, {'game_id': 2017020586, 'season': 20172018, 'game_type': 'R', 'away_team_id': 20, 'home_team_id': 24, 'outcome

In [113]:
team_info_clean.to_sql(name = 'team_info', con = engine, if_exists = 'append', index = False)
player_info_clean.to_sql(name = 'player_info', con = engine, if_exists = 'append', index = False)
goalie_stats_clean.to_sql(name = 'goalie_info', con = engine, if_exists = 'append', index = False)
game_skater_clean.to_sql(name = 'game_skater_info', con = engine, if_exists = 'append', index = False)
game_team_clean.to_sql(name = 'game_team_info', con = engine, if_exists = 'append', index = False)


In [114]:
engine.table_names()

['player_info',
 'team_info',
 'goalie_info',
 'penalty_info',
 'game_skater_info',
 'game_team_info',
 'game_play_info',
 'game_info']

# Query the Database

In [115]:
# Done in pgAdmin 4

